# step by step notebook for reproducing the workflow from RT pred manuscript 

Ensure that the requriements are met i.e.
- rdkit 
- pytorch 
- deepchem
- chemprop
- etc.. 




In [1]:
import pandas as pd 
from rdkit import Chem
from rdkit import RDLogger 
import os 

RDLogger.DisableLog('rdApp.*') # disable rdkit log
seed = 42


## Featurization and data splitting

This workflow is run using the METLIN SMRT Dataset published by Domingo-Almenara et al. https://www.nature.com/articles/s41467-019-13680-7 

But any dataset could be used, however, it must contain columns 'rt' and 'smiles'

By default we remove non-retained compounds from the METLIN SMRT dataset and subsample (2000, randomly)

In [2]:
def get_METLIN_data(sampling = 2000, only_retained = True):
    ''' Function to pull METLIN SMRT data from remote URL and save it as a csv file'''
    #* pulling data from a URL
    url = 'https://figshare.com/ndownloader/files/18130628'

    #* read the data into a pandas dataframe and convert inchi to SMILES
    df = pd.read_csv(url, sep=';')
    df['mol'] = [Chem.MolFromInchi(x) for x in df['inchi']]
    df['smiles'] = [Chem.MolToSmiles(mol) if mol is not None else None for mol in df['mol']]
    df = df.dropna(axis = 0) #* dropping rows with NaN values as some mols did not convert to SMILES

    #* removing non-retained compounds
    if only_retained:
        df = df[df['rt'] > 200]

    #* sampling the data
    df_subset = df[['smiles','rt']].sample(sampling)

    #* outputting the data in a csv file
    output_dir = '../data/metlin_smrt'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    df_subset.to_csv(os.path.join(output_dir, f'sample_dataset_{sampling}_datapoints.csv'), index=False)
    print(df_subset.head(3))

In [3]:
get_METLIN_data(sampling = 100, only_retained = True)

                                                  smiles     rt
59360  OC(=Nc1cccc(Cl)c1)c1cn(C[C@H](O)c2ccc3c(c2)OCO...  863.0
34436  N=C(O)C1CCN(C(=O)c2ccc(CN3C(=O)CCn4nc(-c5ccc(C...  712.7
31869                Cc1cc(=NC(=O)c2ccc(=O)n(C)n2)[nH]o1  597.7


There are a number of features that we use for modeling. 

- LogD descriptor set (pH 0.5 to 7.4)
- ECFP4-2048 fingerprints
- RDKit descriptors 
- molecular graph convolutions 

based on the feature_list ``` feature_list = ['logD', 'ecfp4_csv', 'ecfp4_disk', 'rdkit_csv', 'rdkit_disk', 'molgraphconv]``` it is possible to define which features to be calculated. 

Note that the ECFP4 and RDKit descriptors come in two variants: a flat .csv file or a DeepChem DiskData object. This choice will depend on downstream application and models.


In [19]:
feature_list = ['logD', 'ecfp4_csv', 'ecfp4_disk', 'rdkit_csv', 'rdkit_disk', 'molgraphconv']


import deepchem as dc
                
def get_features(feature_list, path_to_chemaxon_licence = None):
    '''function to get the features from the input feature_list'''
    
    ###* LOG D Calculations
    if 'logD' in feature_list and path_to_chemaxon_licence is not None:
        target_dir = os.path.join(feature_dir,'logd_calculations')
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        ##* the SMILES must be saved in a flat file for the cmdline tool to read them 
        # subsetting the test data, to save time
        data_for_calc = data[['SMILES']].reset_index(drop=True)

        print('ChemAxon-based LogD calculations')

        desc = LogD_calculations(data_for_calc, path_to_binning_matrix=None)

        desc = data.merge(desc, on = 'SMILES', how = 'left')

        desc.to_csv(os.path.join(target_dir, 'all_data.csv'), index=False)


        NA_alert = desc.isna().any().any()
        if NA_alert:
            print('*** NA values in:',target_dir)
        else:
            print('*** No NA found in :', target_dir)
            
            
    ###* ECFP4 Calculations
    if 'ecfp4_disk' in feature_list: 
    
        target_dir = os.path.join(feature_dir,'ecfp4_disk')
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)
            
        print('ECFP4 Featurization - to DiskDataset')

        ## creating a deepchem dataset with featurized smiles
        featurizer = dc.feat.CircularFingerprint(size=nBits, radius=radius)
        feats = featurizer.featurize(data.SMILES)
        dataset = dc.data.DiskDataset.from_numpy(feats, 
                                                data.adj_RT_sec, 
                                                tasks = ['RT_sec'], 
                                                ids = data.SMILES,
                                                data_dir = os.path.join(target_dir,'all_data'))
    

    if 'ecfp4_csv' in feature_list: 
    
        target_dir = os.path.join(feature_dir,'ecfp4_csv')
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        feats_df = dataset.to_dataframe()
        feats_df.to_csv(os.path.join(target_dir, 'all_data_ecfp4.csv'),index = False)

        NA_alert = feats_df.isna().any().any()
        if NA_alert:
            print('*** NA values in:',target_dir)
        else:
            print('*** No NA found in :', target_dir)


ModuleNotFoundError: No module named 'deepchem'